## **Name**:
## **Status**: (e.g., UG3, G1, etc.)
## **Department**:

# ***CBE 512. Machine Learning in Chemical Science and Engineering.***
## **Assignment 05:** *Neural Networks, Cheminformatics, Clustering*
### &#169; Princeton University
### **DUE**: 11:59pm, December 05, 2023
### **Notes**: Supply responses/solutions in a packaged format, including notebook (.ipynb) and any supporting files required to run the notebook. Your assignment should be submitted to Canvas.



---
## **Problem 1, Concept checks (20 points):**

**(a)** Pick two clustering techniques from distinct algorithmic classes (e.g., partition, distribution, hierarchical, density-based). Describe the methdological essence of your chosen techniques. In this description, highlight any requisite hyperparameters as well as prospective advantages and disadvantages.

---


***Student Response:***

**(b)** In class, we described two conventional approaches to molecular featurization. The first is predicated on the generation of *fingerprints* (usually bit vectors), and the second relates to the generation of *vectors of descriptors*. Describe the basic elements of each, commenting on potential advantages or disadvantages as related to property-prediction tasks.

---

***Student Response:***

**(c)** Suppose you are using molecular fingerprints as your featurization approach to molecules. In class, we mentioned multiple ways to think of *similarity* or *dissimilarity* between pairs of molecules. Pick one and describe how it works. What are the challenges with using Euclidean distances to express dissimilarity?

---

***Student Response:***



---


## **Problem 2, a concrete example of neural networks (80 points):**

Examine the contents of `concrete_data.csv` and the description in `readme.concrete_data.csv`.
You can download and view it on your own or easily examine the contents on github [here](https://raw.githubusercontent.com/webbtheosim/CBE512-MLinChmSciEng/main/data/concrete_data.csv) and [here](https://github.com/webbtheosim/CBE512-MLinChmSciEng/blob/main/data/readme.concrete_data.txt).

This is a dataset featuring 1030 experimental measurements that characterize the compressive strength of concrete that is distinguished by 9 material descriptors: 'Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer', 'Coarse Aggregate', 'Fine Aggregate', and 'Age'. The first eight descriptors reflect concentrations of components (kg per cubic meter of mixture) and the last is self-explanatory (units of days). All the data is "raw," so you should conduct your own preprocessing.

Overall, you are going to building and testing neural networks to predict compressive strength of concrete from the materials descriptors. For computational expediency, you may use simple train-test splits in lieu of k-fold cross-validation for reporting.

If you are having trouble accomplishing the tasks with the amount of data provided, you can truncate the dataset as needed (e.g., use 200 points instead of 1000).

---



**(a)** Set up a code to evaluate the dataset complexity. We have 1000+ data points -- are all of these necessary to obtain effective models? Begin by creating an 80/20 train-test split. The test split will be used for consistent evaluation. With the remaining training data, consider randomly sampling different fractional portions of the data (i.e., use 20%, 40%,..., etc.). With each sub-sampling, you will train a neural network. Fix the architecture to be something like [200->100->50->10->1]. Other recommendations would be ReLU activations for all layers except the final layer, the ADAM optimizer, MSE as a loss function, 500 epochs for training, and a batch size of 16; you can modify any of these as you like. Make a plot of test-set performance versus training set size and comment on these results.

---

In [ ]:
# import modules for this problem


**(b)** Technically, the number of epochs for training is an important hyperparameter for model construction. We have often mentioned the approach of "Early Stopping," for which training stops (or returns) back to the state that minimized the validation loss (usually to mitigate subsequent overfitting).

First, make a plot of the training and test set performance versus the number of epochs using the model that used 60% of the total available data for training. Is there a number of epochs for training for which the test loss reaches a minimum? Note that you can specify data for "validation" within the Keras API (rather than specifying the split ratio).

Next, use Keras's callback functionality to implement EarlyStopping and save the model as a .hdf5 format using ModelCheckpoint. For early stopping, you want to monitor `val_loss`, with `mode=min` and `patience=50`. Report what epoch yielded your best model. Should this be the same as what you saw in the previous run manual run? Finally, use load_model to load in your saved best model, and report the MSE/R2 performance of your best early stopping model on the train/test set.

Early stopping implementation is described [here]()

---





In [ ]:
'''
# example of adding a callback
callbackES = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)
# presume model is created in model
model.compile(tf.keras.optimizers.SGD(), loss='mse')
history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5),
...                     epochs=10, batch_size=1, callbacks=[callbackES],
...                     verbose=0)
'''

**(c)** We have not talked significantly about weight initialization for neural networks. During model optimization, we are trying identify the weights that minimize our target loss -- this process is guided through backpropogation. Like most optimizations, success can depend on initial guesses. With neural networks, the process is confounded by the so-called vanishing/exploding gradient problem.

Early work in weight initialization helped mitigate that problem by being smarter about initialization.
Let's explore a little bit about how our weight initialization choices can influence model convergence. Make a bar plot of the train/test performance of your neural network model using three settings for the weight initialization: ["zeros", "RandomUniform", "GlorotUniform","HeNormal"]. Comment on any tangible observations.

The initializers are described [here](https://keras.io/api/layers/initializers/).

---






In [ ]:
'''
# How to assign initializer type to a layer
initializer = tf.keras.initializers.HeNormal()
layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

# add layer to model...
'''

**(d)** We have also discussed dropout regularization as a strategy to mitigate overfitting to the training set. Add a dropout layer after each of the first four neural network layers. Make a bar plot of the train/test performance for five different dropout probabilities: [0.0, 0.1, 0.2, 0.3, 0.4]. What do you observe about the performance on the training and test sets, respectively, as the dropout rate increases? Is there a best dropout rate for your model?

*Remember that during testing, the dropout layers should be removed!*

---



---

## **Problem 3. Examining a chemical landscape. (40 points):**

In this problem we are going to try out some dimensionality reduction and clustering. We are going to do this using a fraction of the [ChEMBL database](https://www.ebi.ac.uk/chembl/), which features bioactive compounds. We have extracted a subset of 1500 molecules from this database and hosted them on our [github](https://raw.githubusercontent.com/webbtheosim/CBE512-MLinChmSciEng/main/data/chembl_1500_subset.smi) (take a look, it is named `chembl_1500_subset.smi`). The `.smi` is human-readable; the first column features a smiles string and the second column is a molecule code.

---



**(a)** Download and process `chembl_1500_subset.smi`. Using a cheminformatics package, convert the smiles strings to a set of feature vectors/fingerprints. One recommended possibility is the use of MACCSkey fingerprints (~160-dimensional based on SMARTS patterns). If using an alternative fingerprint, it is recommended that you generate vectors that are  less than 200-dimensional.

You may find it particularly useful to use [deepchem](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html).

Make a plot that illustrates the frequency of occurrence of each bit across the set of molecules extracted.

---



**(b)** Utilize a dimensionality reduction technique to create a two-dimensional projection of the extracted chemical space. You can use a linear technique or a [non-linear manifold learning technique](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold). Often, linear techniques (like PCA) are actually applied as a preprocessing step prior to deploying non-linear techniques that may be computationally expensive. Visualize your results.

*If you are having trouble accomplishing the task on your machine, you can further truncate the dataset by subsampling just a fraction*.

---

**(c)** Select a clustering technique of interest, such as k-means or DBSCAN, and perform clustering in the projection-space obtained from the unsupervised learning algorithm. Study how clustering results are affected by changing hyperparameters associated with the clustering technique and monitoring the silhouette score. Are there natural clusters in the data/do the clusters make sense?

It may be useful to select a specific cluster and a cluster far away to inspect whether molecules within one cluster seem intuitively more related than ones in the distant cluster. Without doing so (although you may if you want), how could you further quantify whether intra-cluster chemical similarity is higher than between-group chemical similarity apart from the [cluster-based metrics](https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation) themselves?

---